In [3]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
from urllib.request import urlopen
import re
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools

Schwab_BASE = 'https://www.schwabjobs.com'
PNC_BASE = 'https://www.indeed.com/viewjob?'

PNC_URL = "https://www.indeed.com/jobs?q=Pnc+Bank"
Schwab_URL = "https://www.schwabjobs.com/List/Custom/Schwab-Job-Category"

stateList = []

banks = ["PNC","Schwab"]
bank = ''
listId = 1
count = 1
BASE = 'https://www.indeed.com/'
FIRST_LISTING_PAGE = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?clearPrefilter=1'
LISTING_URL_PREFIX = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start='
LISTING_URL_POSTFIX = 'clearPrefilter=1#cmp-menu-container'

def keywordsFileName(listId):
    if listId == 1:
        return 'frequency.csv'
    elif listId == 2:
        return 'tfidf.csv'
    else:
        return 'textrank.csv'

def scrape(postingUrl):
    print(postingUrl)
    global count
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)", "List id (1,2,3)"]

    values = [count, bank, listId, postingUrl, listId]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open(keywordsFileName(listId), 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        request = urllib2.Request(postingUrl,headers=hdr)
        html_page = urlopen(request)
        soup2 =  BeautifulSoup(html_page,"html.parser")
        if bank == 'PNC Bank':
            value = soup2.find('input', {'id':'preLoadJSON'})
            for word in value.get('value').split('"CandLandPageText"')[1].replace("\\u003", " ").split():
                if(word in arr):
                    ind = arr.index(word)
                    wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
            dataDict = postingDict.copy()  
            return {**postingDict, **wordCountDict}
                
        else:
            value = soup2.findAll("div", {"class": "desc"})
            for val in value:
                st = val.text.split()
                for word in st:
                    if(word in arr):
                        ind = arr.index(word)
                        wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
    except:
        return ("An exception occurred")
    
    count = count + 1

    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 
 
def scrape_page_pnc(page):
    response = urlopen(page)
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        links = bs_obj.findAll('div',{'class': 'cmp-section cmp-with-border'})
        for page in links:
            linkTags = page.find_all('a', attrs={'href': re.compile("/cmp/_")})
            for tag in linkTags:
                link = tag.get('href')
                postingUrl = BASE + link
                pageDict = scrape(postingUrl)
                if (type(pageDict) is dict):
                        data = pd.DataFrame([pageDict])
                        # if file does not exist write header 
                        if not os.path.isfile('data.csv'):
                            data.to_csv('data.csv', header='column_names')
                        else: 
                            # else it exists so append without writing the header
                            data.to_csv('data.csv', mode='a', header=False)
                else:
                    print("exception?")
    


def scrape_pages_schwab(pages):
    stateList = {}
    for p in pages:
        pageHtml = urlopen(p)
        soup = BeautifulSoup(pageHtml)
        table = soup.find('table', 'JobListTable')
        pageDataframe = pd.DataFrame([])
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = Schwab_BASE + link.get('href')
            state = table.findAll("td",{"class": "colstate"})
            for s in state:
                st = re.sub(r'[^ \w\.]', '', s.text)
                if st not in stateList:
                    stateList[st] = 1
                else:
                    stateList[st] += 1
            pageDict = scrape(page)
            if (type(pageDict) is dict):
                data = pd.DataFrame([pageDict])
                # if file does not exist write header 
                if not os.path.isfile('data.csv'):
                    data.to_csv('data.csv', header='column_names')
                else: # else it exists so append without writing the header
                    data.to_csv('data.csv', mode='a', header=False)
    with open('states.csv', 'a+', encoding = 'utf-8') as f: 
        for key in stateList.keys():
            f.write("%s,%d\n"%(key,stateList[key]))
        

def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(Schwab_BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(Schwab_BASE + key)
    return pageLinks

def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []
        cat = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
                cat.append(tag.text)
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        #categCsv = dict(zip(cat, numPages))
        #plt.bar(range(len(categCsv)), list(categCsv.values()), align='center')
        #plt.xticks(range(len(categCsv)), list(categCsv.keys()))
        #plt.imshow()
        return dict(zip(catLinks, numPages))

def scrape_site(Url):
    scrape_page_pnc(FIRST_LISTING_PAGE)
    for i in range(12,996,12):
        nextPageUrl = LISTING_URL_PREFIX +  str(i) + "&" + LISTING_URL_POSTFIX
        scrape_page_pnc(nextPageUrl)

def get_data ():
    if (bank == "PNC"):
        scrape_site(PNC_URL)
    elif (bank == "Schwab"):
        res = urllib2.urlopen(Schwab_URL)
        categories = get_category(res)
        pages = get_pages(categories)
        scrape_pages_schwab(pages)

if __name__ == '__main__':
    if os.path.isfile('data.csv'):
        os.remove('data.csv')
    for i in range(1,3):
        listId = i
        print(listId)
        for b in banks:
            bank = b
            print(bank)
            get_data()

1
PNC
https://www.indeed.com//cmp/_/job?jk=4863a5c07b0714ae&tk=1d82gbb2q52ta800&from=highlightedJob&rd=n5r6n1S2hixwO8gpEj_fuH-oTfDWiB7OFvaWK58rLaqEWKnzkB3CzayxYMOzK2x1hqR7wF73N1wV0PeXjxNZKbTJPyF98XI80Ts6aKQ45j-e_mWA9mO_9EtVcyXrUw_jaKQSqks1ui-zzCiPToSwUvj7soM2Nsw7hPRgIHgVE7pjt5mE6eUOr-VTIo39RrX0EgltnhWel1XvsC_unl5Oi7cfZLmd8i1dPkPKl25kUlSSmoy2MbrpD1KU94_LjeZwWnnI1PtU00DNUP233gEBxJ78jFCgm0lPX-qJJ7fcKuSzsBzM_oqSVaNodR7SdCzoZR7FW1b608ME6qRJU1nLPlHFoxpnQN8nZ4ujy2c1-Y9Su_FaSGYTBYYBEhSNlAe7YrecSWM8fiELvJz6HUGM4ghCgZClyxyKS4BWgCDfwylDt5aH1j5VrUEH9Sy-5sNLmvvCSGU0nJMqe2VFX19RfE000-hpRQNaRnvIYG2W3lo9iGkQXMAg5_kadDFQ8ZUfCrFqCGrYAY0Q5z0M2G8g3D4X84dSuP0GfQYHpW_myrKlDSj87um2TVBLE0Vbqtf8V2rAWJOLa2AEsVChATJLm49HH5TOJiUVX59yXZwAyccYxaAIlRmSTONtcXJYUyLc6ELM2Bjrz_2BdUcn-iE5SmEOB5JnwTZWHSSjmqhBzdpBD0XlHuv3J0BYzFL6oir3Qu-9pMwM8-SsVP7F-KgBGdk3kEVW-aYKHRb5xsov065MDtwzNJZaiFpGM8pVfNPaSxze_zTVhuosGcYSJpLyTO7yMIqnDLbXWokn0FYkVFf6uEWcqhXiAdssDHv3_yit2MQt1Gsz9Vcqm4uvkZRNOQuTOfN3yaPP-zyoPqbroZK_iVjyJZQepzxOGYlUyCPHg

exception?
https://www.indeed.com//cmp/_/job?jk=7a3586572797487d&tk=1d82gbn6b0mtn000
exception?
https://www.indeed.com//cmp/_/job?jk=0564d498a1cd9b58&tk=1d82gbn6b0mtn000
exception?
https://www.indeed.com//cmp/_/job?jk=f276d2e5da0ac2d1&tk=1d82gbn6b0mtn000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXadHB6vtsAwbgswve1R9-ZKajcCXeo6yCfO4FpAvyHQdVr_vxMovTdmuiEg_paqgQNf66QpWkwYcoL6-ey7JfHdR1rrPfPiqtaNA3uhSgKixzgN1QX6XkPoUSVonUNaS_mFd8PjneX0fH7zJr9_vcFnEgtz360cAXgo4ENqiDAG-exO9-EFx-UrBVq6AJKfdiBDsnDz7Un4Sms3vjKCdbug4Pzgx1C8jNFvV73l-zjXNAk3fsChw69Td4Sytp3uk2iqoIqC4gIzGZn737dRhIZ_JCoHtAzsbaVTPujVByE5-xRSb3tIpHDK-fDGXFCJwVpKOx2PAehrdfNJMG-bTFf2C_GPNTVxfuskKaWFxqa-cXpu6mDjteZXofLqPKd8RGM24mqSndwXGW5h2sNq6oDAlI7uHijFWNaOA78xil4OUHr5xbgxIap9TrvrWv9GGodMd8k2xteAgfpWbmIiKNZVqUTYuu6OjonlGV5SHb2aASKJfaGzgoCukYM6RG_mZlXQ
exception?
https://www.indeed.com//cmp/_/job?jk=7a489537800da7eb&tk=1d82gbn6b0mtn000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yu

exception?
https://www.indeed.com//cmp/_/job?jk=4863a5c07b0714ae&tk=1d82gcfls1ba9000&from=highlightedJob&rd=n5r6n1S2hixwO8gpEj_fuH-oTfDWiB7OFvaWK58rLaqEWKnzkB3CzayxYMOzK2x1hqR7wF73N1wV0PeXjxNZKbTJPyF98XI80Ts6aKQ45j-e_mWA9mO_9EtVcyXrUw_jaKQSqks1ui-zzCiPToSwUvj7soM2Nsw7hPRgIHgVE7pjt5mE6eUOr-VTIo39RrX0EgltnhWel1XvsC_unl5Oi7cfZLmd8i1dPkPKl25kUlRV6Ws_sqn5ISivBsjqdOmEtMzm5ViMOPuOG_Fq6toyK3uDvk85CR4bnLXtIVbO4rOGYixp47HkGBQ6Frmx9xA94tsKvjW6Rn2hsxTza1-pcaBfRrWkI0jA4OxXXWW8JxS6zjA99YrLz_TXgoVIUL88IQFzN6oH9DRPkeED04ctVcugl9Zg9YqPaMIdGXfWmVjFcQhyl5fc77YDZ_4olcddB2g7dc7Q5EsdWcTBEJ1xBA2m86rrg3cQ61M3hxfq4jiEzY3Ln8IkZdU4Cbwsi3LE2sT4zWjxeMNgutC9wGnfjmmOKeKREkHsKG3ksxGUeoK_4GM5ITmH_hRnEuPGKre7mAVlK6jEYdkkouUPGrq6M47nPghq3RR1mxsjjG1FalUn1dsPcS6dRGkE37cOULUO2tt7iDWtQgccNAOrSs9Uj_aogcT7TRt62eoCOR_NYWIBhW0cQ0vU5C_VqXK9jSdm0AoFpDyD-OAHZcSi2bXxesBLMOosN1Ewpu9EBNYLVqocdxIEYbV0TDX-U96MQMQTdigWhVBNMTUrGF0VtGtU_lO1D_BIpZoBRreriMZjfHSHoyhjYO1W9ln9HHcHtWcYy2OXXaCbwxysJ72gO2-3K072AsDB45Y68zrImwaobW5gegWozdZWeKa9uHj9

exception?
https://www.indeed.com//cmp/_/job?jk=65d646fdd7fa3269&tk=1d82gcrk60mtn000
exception?
https://www.indeed.com//cmp/_/job?jk=24a596f01cb2429c&tk=1d82gcrk60mtn000
exception?
https://www.indeed.com//cmp/_/job?jk=07aae2d0a6284140&tk=1d82gcrk60mtn000
exception?
https://www.indeed.com//cmp/_/job?jk=1767a066400f1f66&tk=1d82gcrk60mtn000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXafOcsJpyjjvr3CeSwPVvrNtppoopQ_oVvkT2AWHtSe6-tOFTxSedcoNpPYxZhF7_0KPEuDkDEy6UcbtdWfoPDKfOLnSe3254ZUznyV8fnnpiu40Sn9v3bLRrWpMrJdUt4I7qdDcvySmM3VpS7zgCrAf1Z4OHi0o4da93kZ9sI5acqQGgDgOa_kVZ92xiwYdWPEV_6bhM6rEGcvL6GhOGf2WARC9AI3BAWEMr2L_Q9OzNyPECyITJHhaFveYa_fvNGdCxxc9w6JWjjH_kfZHzHLYvwgsZFcMT9WvZcdCdlh6Zl6dU6pF0ujQSNOmZU7kQCTqekvLGztb2XeYzgHoBXkdXn2TJQvoAmlnA3PPMpX4pfxfCoBfONvruaJ4KZ4BdsNLg6tSE-xUDUCf2BYM_GaZ9AHEYMZsycOSPOOMW8PQdYBue6f3NyX_smuttkeR8uxjGJhgVqcZKUW_Ma7EhtkQoQ9U-SbEbLMvJkr8vnyTFdQGcglWHRthmQC172TxpRk
exception?
https://www.indeed.com//cm

exception?
https://www.indeed.com//cmp/_/job?jk=3545506b6b7c7832&tk=1d82gd6n952ta800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXafOcsJpyjjvr3CeSwPVvrNtHtPxMkuppOHCHaEfPuNDqKu1CpBg-EXgrfgpx_jgqmgnlzoZTVEW1Bbr7z6dP3-2IQIQREogqvpFrGTJERr6d4lW8lPwLS7bXHL4P14gsK2-cWvi-rbf-jnZtGC07ClwFpBXTY-PBZ6BUAWcYPQJY17LerLtY9lXxCz9cZvq8aQ842uVkQ0zCpCgdnD6VExtqX71v2wy30A3VNz8RU5TQuZxWonKeI9DJogdOfWXrapAVvz0ZYYhYGhtl41nXSKPa_Wq5dM4Qc0ymzVsHf_RjVEMeiGi3GuxEhVxj3lIyoBzMdmcs6i7VJ1o9fZ41tcOdXbNFrZ3ysmMguV2-Yp2VsRAb_nwjDvtQv-sOW7kRW79zvRsfY5PfczYOh_IieaxF4HO5MwbuGEfaHYTxL6w1BME1HZqGUWpQLB8zG0sjeQSQEvLk-1660gf00BzRZJZmYxqA6ykTvYSgmeFfHX4BpEWPm1IB2AJ2Lxe96DsX4A
exception?
https://www.indeed.com//cmp/_/job?jk=4863a5c07b0714ae&tk=1d82gdi3v0mt6000&from=highlightedJob&rd=n5r6n1S2hixwO8gpEj_fuH-oTfDWiB7OFvaWK58rLaqEWKnzkB3CzayxYMOzK2x1hqR7wF73N1wV0PeXjxNZKbTJPyF98XI80Ts6aKQ45j-e_mWA9mO_9EtVcyXrUw_jaKQSqks1ui-zzCiPToSwUvj7soM2Nsw7hPRgIHgVE7pjt5mE6eUOr-VT

...

https://www.schwabjobs.com/ShowJob/Id/1997687/Mobile-App-Product-Owner/
https://www.schwabjobs.com/ShowJob/Id/1998646/Senior-Product-Manager/
https://www.schwabjobs.com/ShowJob/Id/1998965/Senior-Product-Manager,-Digital-Platform-Solutions/
https://www.schwabjobs.com/ShowJob/Id/1998977/Sr.-Manager,-Digital-Product-Management/
https://www.schwabjobs.com/ShowJob/Id/2000796/Tech-Director-–-Innovation-Accelerator/
https://www.schwabjobs.com/ShowJob/Id/2016914/Conference-Coordinator/
https://www.schwabjobs.com/ShowJob/Id/1896376/Sr.-Manager,-Construction-Project-Manager/
https://www.schwabjobs.com/ShowJob/Id/1939044/Data-Center-Facilities-Manager/
https://www.schwabjobs.com/ShowJob/Id/1994128/Regional-Property-Manager/
https://www.schwabjobs.com/ShowJob/Id/2018616/Financial-Consultant-Partner-Phoenix,-AZ/
https://www.schwabjobs.com/ShowJob/Id/1957436/VP,-Financial-Consultant-Paramus,-NJ/
https://www.schwabjobs.com/ShowJob/Id/2018303/Associate-Financial-Consultant-Tampa,-FL/
https://www.schwa

https://www.schwabjobs.com/ShowJob/Id/1987927/Financial-Consultant-Partner-Hong-Kong/
https://www.schwabjobs.com/ShowJob/Id/2005363/Private-Client-Advisor/
https://www.schwabjobs.com/ShowJob/Id/2006892/Associate-Private-Client-Advisor/
https://www.schwabjobs.com/ShowJob/Id/1995794/Private-Client-Advisor/
https://www.schwabjobs.com/ShowJob/Id/2004132/Investment-Consultant/
https://www.schwabjobs.com/ShowJob/Id/2005856/HR-Reporting-Analyst-(HRIS-Analyst)/
https://www.schwabjobs.com/ShowJob/Id/1993517/Sr.-Manager,-HR-Business-Partner-Technology/
https://www.schwabjobs.com/ShowJob/Id/1998381/Learning-Manager/
https://www.schwabjobs.com/ShowJob/Id/2018851/Staff-IAM-Operations-Security-Analyst/
https://www.schwabjobs.com/ShowJob/Id/2018814/Software-Developer-in-Test-(SDET)/
https://www.schwabjobs.com/ShowJob/Id/1950614/Sr.-Software-Development-Manager-C#-.Net/
https://www.schwabjobs.com/ShowJob/Id/2018577/Staff-Software-Engineer/
https://www.schwabjobs.com/ShowJob/Id/2018572/Managing-Directo

https://www.schwabjobs.com/ShowJob/Id/1959354/Security-Analyst-IAM/
https://www.schwabjobs.com/ShowJob/Id/1966387/Sr.-Security-Analyst-IAM/
https://www.schwabjobs.com/ShowJob/Id/1938455/Sr.-Staff-of-Technology-Risk-Governance/
https://www.schwabjobs.com/ShowJob/Id/1710623/Staff,-Java-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1956085/Software-Engineer-Drupal/
https://www.schwabjobs.com/ShowJob/Id/1937897/Senior-Staff-Engineer-Senior-.Net-Developer/
https://www.schwabjobs.com/ShowJob/Id/2007064/Managing-Director,-Infrastructure-Architecture/
https://www.schwabjobs.com/ShowJob/Id/2007063/Managing-Director,-Software-Application-Engineering/
https://www.schwabjobs.com/ShowJob/Id/1964129/Senior-CI-CD-Release-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1992512/Sr.-Staff-Scrum-Master/
https://www.schwabjobs.com/ShowJob/Id/2006210/Sr-Data-Engineer/
https://www.schwabjobs.com/ShowJob/Id/2005854/Staff-Software-Development-Engineer-Java/
https://www.schwabjobs.com/ShowJob/Id/1761042/Solu

https://www.schwabjobs.com/ShowJob/Id/1926217/Sr-Manager-Data-Engineering/
https://www.schwabjobs.com/ShowJob/Id/1852619/Senior-Android-Developer/
https://www.schwabjobs.com/ShowJob/Id/1910920/Sr-Software-Developer,-C#.Net/
https://www.schwabjobs.com/ShowJob/Id/1911516/Senior-Teradata-Administrator/
https://www.schwabjobs.com/ShowJob/Id/1912146/Sr.-Manager,-Software-Applications-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1917580/Sr.-Technical-Solutions-Architect/
https://www.schwabjobs.com/ShowJob/Id/1917582/Sr.-Technical-Solutions-Architect/
https://www.schwabjobs.com/ShowJob/Id/1922626/ITSM-Configuration-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1935241/Sr-Managing-Director-Software-Ap-Engineering-.Net-(AST)/
https://www.schwabjobs.com/ShowJob/Id/2004133/Sr.-Staff-Cyber-Resilience-Oversight/
https://www.schwabjobs.com/ShowJob/Id/2001589/Program-Manager,-Business-Intelligence/
https://www.schwabjobs.com/ShowJob/Id/2002933/SQA-Architect,-Technical-Director/
https://www.schwab

https://www.schwabjobs.com/ShowJob/Id/1987137/Relationship-Manager,-Retirement-Plan-Services/
https://www.schwabjobs.com/ShowJob/Id/1988256/Internal-Control-Risk-Management-Manager/
https://www.schwabjobs.com/ShowJob/Id/1995799/Communications-Consultant/
https://www.schwabjobs.com/ShowJob/Id/1996053/Operations-Specialist/
https://www.schwabjobs.com/ShowJob/Id/1996570/Business-Analyst-Retirement-Plan-Services/
https://www.schwabjobs.com/ShowJob/Id/1997741/Client-Services-Team-Manager/
https://www.schwabjobs.com/ShowJob/Id/2001058/Client-Service-Manager-Academy/
https://www.schwabjobs.com/ShowJob/Id/1961228/Sr-Manager-Operational-Risk-Testing/
https://www.schwabjobs.com/ShowJob/Id/2016590/Director,-Operational-Risk-Management-Liaison-Team/
https://www.schwabjobs.com/ShowJob/Id/1979047/Senior-Manager,-Finance-Central-Risk-Management/
https://www.schwabjobs.com/ShowJob/Id/1910923/Manager-eDART-Risk-Governance-Business-Integration/
https://www.schwabjobs.com/ShowJob/Id/1991653/Fraud-Strateg